In [94]:
import numpy as np
import pandas as pd 
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score


In [6]:
np.random.seed(110243143)
data = np.loadtxt('EMG.csv',skiprows=1,delimiter=',')

In [38]:
pdData = pd.read_csv('EMG.csv')
pdData.head()

,S1_0,S2_0,S3_0,S4_0,S5_0,S6_0,S7_0,S8_0,S1_1,S2_1,...,S8_6,S1_7,S2_7,S3_7,S4_7,S5_7,S6_7,S7_7,S8_7,Target
0,26.0,4.0,5.0,8.0,-1.0,-13.0,-109.0,-66.0,-9.0,2.0,...,-28.0,61.0,4.0,8.0,5.0,4.0,-7.0,-59.0,16.0,0
1,-47.0,-6.0,-5.0,-7.0,13.0,-1.0,35.0,-10.0,10.0,-4.0,...,-25.0,47.0,6.0,6.0,5.0,13.0,21.0,111.0,15.0,0
2,-19.0,-8.0,-8.0,-8.0,-21.0,-6.0,-79.0,12.0,0.0,5.0,...,-83.0,7.0,7.0,1.0,-8.0,7.0,21.0,114.0,48.0,0
3,2.0,3.0,0.0,2.0,0.0,22.0,106.0,-14.0,-16.0,-2.0,...,-38.0,-11.0,4.0,7.0,11.0,33.0,39.0,119.0,43.0,0
4,6.0,0.0,0.0,-2.0,-14.0,10.0,-51.0,5.0,7.0,0.0,...,38.0,-35.0,-8.0,2.0,6.0,-13.0,-24.0,-112.0,-69.0,0


In [82]:
y = pdData.Target
print(y)
X = pdData.drop('Target', axis=1)

0        0
1        0
2        0
3        0
4        0
        ..
11673    3
11674    3
11675    3
11676    3
11677    3
Name: Target, Length: 11678, dtype: int64


In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
X_test

,S1_0,S2_0,S3_0,S4_0,S5_0,S6_0,S7_0,S8_0,S1_1,S2_1,...,S7_6,S8_6,S1_7,S2_7,S3_7,S4_7,S5_7,S6_7,S7_7,S8_7
2842,-2.0,0.0,-2.0,-1.0,-10.0,-15.0,25.0,-16.0,-1.0,3.0,...,57.0,22.0,-14.0,-4.0,-2.0,1.0,19.0,2.0,-30.0,-3.0
1193,7.0,4.0,-6.0,-8.0,4.0,6.0,18.0,23.0,-48.0,4.0,...,-31.0,-28.0,-10.0,14.0,2.0,-1.0,-7.0,-43.0,-40.0,-4.0
11578,3.0,0.0,-2.0,-6.0,0.0,12.0,4.0,-13.0,1.0,2.0,...,12.0,0.0,-3.0,-2.0,-1.0,4.0,17.0,-19.0,-8.0,1.0
9338,8.0,3.0,3.0,6.0,14.0,4.0,5.0,2.0,0.0,-4.0,...,-5.0,-6.0,-3.0,0.0,0.0,-4.0,0.0,-19.0,-10.0,-10.0
4015,-7.0,-5.0,-6.0,-9.0,6.0,9.0,-1.0,-1.0,-8.0,-8.0,...,-1.0,4.0,-33.0,0.0,3.0,3.0,-64.0,-82.0,-10.0,-17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8842,6.0,-1.0,0.0,2.0,-9.0,8.0,-3.0,1.0,3.0,1.0,...,3.0,-9.0,41.0,4.0,2.0,6.0,4.0,12.0,-13.0,7.0
7938,16.0,-5.0,-3.0,-2.0,3.0,0.0,3.0,-7.0,-28.0,-5.0,...,-2.0,-10.0,14.0,-3.0,-4.0,5.0,26.0,14.0,11.0,21.0
10982,10.0,0.0,1.0,-2.0,-21.0,-35.0,-4.0,1.0,-1.0,3.0,...,-6.0,-11.0,2.0,4.0,4.0,3.0,-13.0,22.0,9.0,12.0
6077,-7.0,-6.0,-3.0,-2.0,-7.0,-18.0,-2.0,0.0,6.0,9.0,...,-2.0,-12.0,13.0,0.0,4.0,6.0,-13.0,-7.0,-1.0,8.0


In [84]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()

In [95]:
accuracy_score(y_train, y_test)




0.24850145572872068

In [97]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_train, y_test)

ValueError: multi_class must be in ('ovo', 'ovr')